In [1]:
import pandas as pd
import re
from jiwer import wer
from jiwer import compute_measures
import ast


## Load Results

In [2]:
kelechi_phi_whisper = pd.read_excel('results/bio_ramp_asr_results.xlsx',engine='openpyxl')
ashisa_results_1 = pd.read_csv('results/afrispeech_asr - afrispeech_asr.csv')
aisha_results_2 = pd.read_csv('results/us_medical_20_asr - us_medical_20_asr.csv')
primock_phi_whisper = pd.read_csv('/home/kelechi/bio_ramp_asr/results/phi_4_asr_results_primock.csv')
primock_granite_parakeet = pd.read_csv('/home/kelechi/bio_ramp_asr/results/primock_asr - primock_asr.csv')

# concat the two results
aisha_granite_parakeet = pd.concat([ashisa_results_1, aisha_results_2], ignore_index=True, sort=False)

evaluate_data = pd.merge(kelechi_phi_whisper, aisha_granite_parakeet[['utterance_id', 'Nvidia-Parakeet', 'IBM-Granite']], on='utterance_id', how='inner')
evaluate_data['Phi-4-ASR'] = evaluate_data['Phi-4-ASR'].apply(lambda x: pd.NA if isinstance(x, str) and "ERROR: CUDA out of memory" in x else x)

# Merge Primock results
primock_merged = pd.merge(primock_phi_whisper, primock_granite_parakeet[['utterance_id', 'Nvidia-Parakeet-doctor', 'Nvidia-Parakeet-patient', 'IBM-Granite-doctor', 'IBM-Granite-patient']], on='utterance_id', how='inner')

## Text Normalize

In [3]:
def remove_timestamps(text: str) -> str:
    """
    Remove timestamps matching patterns like:
      - 03:18:98
      - 00:00:001
    (pattern: 1-2 digits ':' 1-2 digits ':' 1-3 digits)

    Returns cleaned string with extra spaces collapsed.
    """
    if not isinstance(text, str):
        return text
    # remove timestamp tokens
    cleaned = re.sub(r'\b\d{1,2}:\d{1,2}:\d{1,3}\b', '', text)
    # remove new line characters (convert to spaces), then collapse multiple spaces and trim
    cleaned = cleaned.replace('\n', ' ').replace('\r', ' ')
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # remove speaker tags like: [Speaker 1]:  Speaker 1:  speaker1:  D:  P:
    cleaned = re.sub(r'\[?[Ss]peaker\s*\d+\]?:', '', cleaned)
    cleaned = re.sub(r'\b[Dd]:', '', cleaned)
    cleaned = re.sub(r'\b[Pp]:', '', cleaned)
    
    # remove all characters that is not A-Z, a-z, 0-9 or space
    cleaned = re.sub(r"[^a-zA-Z0-9\s]", "", cleaned)
    
    # lowercase the text
    cleaned = cleaned.lower()
    return cleaned


In [4]:
# apply remove_timestamps to the human transcript column
evaluate_data['norm_human_transcript'] = evaluate_data['human-transcript'].apply(remove_timestamps)
evaluate_data['norm_whisper_asr'] = evaluate_data['Whisper-ASR'].apply(remove_timestamps)
evaluate_data['norm_phi4_asr'] = evaluate_data['Phi-4-ASR'].apply(remove_timestamps)
evaluate_data['norm_parakeet'] = evaluate_data['Nvidia-Parakeet'].apply(remove_timestamps)
evaluate_data['norm_granite'] = evaluate_data['IBM-Granite'].apply(remove_timestamps)

In [13]:
primock_merged['norm_human_doctor'] = primock_merged['doctor_utterances'].apply(remove_timestamps)
primock_merged['norm_human_patient'] = primock_merged['patient_utterances'].apply(remove_timestamps)
primock_merged['norm_phi4_doctor'] = primock_merged['Phi-4-ASR-Doctor'].apply(remove_timestamps)
primock_merged['norm_phi4_patient'] = primock_merged['Phi-4-ASR-Patient'].apply(remove_timestamps)
primock_merged['norm_whisper_doctor'] = primock_merged['Whisper-ASR-Doctor'].apply(remove_timestamps)
primock_merged['norm_whisper_patient'] = primock_merged['Whisper-ASR-Patient'].apply(remove_timestamps)
primock_merged['norm_granite_doctor'] = primock_merged['IBM-Granite-doctor'].apply(remove_timestamps)
primock_merged['norm_granite_patient'] = primock_merged['IBM-Granite-patient'].apply(remove_timestamps)
primock_merged['norm_parakeet_doctor'] = primock_merged['Nvidia-Parakeet-doctor'].apply(remove_timestamps)
primock_merged['norm_parakeet_patient'] = primock_merged['Nvidia-Parakeet-patient'].apply(remove_timestamps)

## Calculate WER

In [5]:
norm_columns = ['norm_whisper_asr', 'norm_phi4_asr', 'norm_parakeet', 'norm_granite']
# Calculate WER for each ASR column
for norm_col in norm_columns:
    evaluate_data[f'{norm_col}_wer_compute'] = evaluate_data.apply(
        lambda row: compute_measures(
            row['norm_human_transcript'] if pd.notna(row['norm_human_transcript']) else "",
            row[norm_col] if pd.notna(row[norm_col]) else ""
        ), axis=1
    )
    evaluate_data[f'{norm_col}_wer'] = evaluate_data[f'{norm_col}_wer_compute'].apply(lambda measures: measures['wer'])
    evaluate_data[f'{norm_col}_ins'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['insertions']
    )
    evaluate_data[f'{norm_col}_del'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['deletions']
    )
    evaluate_data[f'{norm_col}_sub'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['substitutions']
    )
    evaluate_data[f'{norm_col}_ops'] = evaluate_data[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['ops']
    )
    

In [14]:
# apply wer calculation to primock as well
primock_norm_columns = ['norm_whisper_doctor', 'norm_whisper_patient', 'norm_phi4_doctor', 'norm_phi4_patient',
                        'norm_granite_doctor', 'norm_granite_patient', 'norm_parakeet_doctor', 'norm_parakeet_patient']
for norm_col in primock_norm_columns:
    primock_merged[f'{norm_col}_wer_compute'] = primock_merged.apply(
        lambda row: compute_measures(
            row['norm_human_doctor'] if 'doctor' in norm_col else row['norm_human_patient'],
            row[norm_col] if pd.notna(row[norm_col]) else ""
        ), axis=1
    )
    primock_merged[f'{norm_col}_wer'] = primock_merged[f'{norm_col}_wer_compute'].apply(lambda measures: measures['wer'])
    primock_merged[f'{norm_col}_ins'] = primock_merged[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['insertions']
    )
    primock_merged[f'{norm_col}_del'] = primock_merged[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['deletions']
    )
    primock_merged[f'{norm_col}_sub'] = primock_merged[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['substitutions']
    )
    primock_merged[f'{norm_col}_ops'] = primock_merged[f'{norm_col}_wer_compute'].apply(
        lambda measures: measures['ops']
    )

## Add Alignment for Sub, Del and Ins Words

In [6]:
def preprocess_alignment_ops(alignment_ops):
    # If already a list/dict, return as-is (may be nested list)
    if isinstance(alignment_ops, (list, dict)):
        return alignment_ops
    if alignment_ops is None or (isinstance(alignment_ops, float) and pd.isna(alignment_ops)):
        return []
    s = str(alignment_ops)
    s = s.replace("AlignmentChunk(", "{").replace(")", "}").replace("type=", "'type':")\
         .replace("ref_start_idx=", "'ref_start_idx':").replace("ref_end_idx=", "'ref_end_idx':")\
         .replace("hyp_start_idx=", "'hyp_start_idx':").replace("hyp_end_idx=", "'hyp_end_idx':")
    if s.startswith("[[") and s.endswith("]]"):
        s = s[1:-1]
    return s

def _get_field(op, field, default=None):
    # support dict-like
    if isinstance(op, dict):
        return op.get(field, default)
    # support object with attribute (AlignmentChunk)
    if hasattr(op, field):
        return getattr(op, field, default)
    # support tuple/list with numeric positions (fallback not used here)
    return default

def extract_words_from_alignment(ref_text, hyp_text, alignment_ops):
    deletions, insertions, substitutions, equals = [], [], [], []
    ref_words = ref_text.split() if isinstance(ref_text, str) else []
    hyp_words = hyp_text.split() if isinstance(hyp_text, str) else []

    # if hypothesis empty -> all deleted
    if not hyp_words:
        return ref_words, [], [], []

    processed = preprocess_alignment_ops(alignment_ops)

    # produce a flat list of op entries regardless of input shape
    if isinstance(processed, list):
        # flatten one level (handles [[...]] case)
        flat = []
        for item in processed:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)
        alignment_ops_list = flat
    elif isinstance(processed, dict):
        alignment_ops_list = [processed]
    else:
        if not processed:
            return deletions, insertions, equals, substitutions
        try:
            alignment_ops_list = ast.literal_eval(processed)
            # if parsed to nested list, flatten one level
            if isinstance(alignment_ops_list, list) and any(isinstance(x, list) for x in alignment_ops_list):
                flat = []
                for item in alignment_ops_list:
                    if isinstance(item, list):
                        flat.extend(item)
                    else:
                        flat.append(item)
                alignment_ops_list = flat
        except Exception as e:
            raise ValueError(f"Failed to parse alignment_ops for row: {e}")

    for op in alignment_ops_list:
        typ = _get_field(op, "type")
        if typ == "delete":
            s = _get_field(op, "ref_start_idx", 0)
            e = _get_field(op, "ref_end_idx", 0)
            deletions.extend(ref_words[s:e])
        elif typ == "equal":
            s = _get_field(op, "hyp_start_idx", 0)
            e = _get_field(op, "hyp_end_idx", 0)
            equals.extend(hyp_words[s:e])
        elif typ == "insert":
            s = _get_field(op, "hyp_start_idx", 0)
            e = _get_field(op, "hyp_end_idx", 0)
            insertions.extend(hyp_words[s:e])
        elif typ == "substitute":
            s = _get_field(op, "ref_start_idx", 0)
            e = _get_field(op, "ref_end_idx", 0)
            substitutions.extend(ref_words[s:e])

    return deletions, insertions, equals, substitutions

In [7]:
# apply extract_words_from_alignment to each row in the DataFrame i.e Whisper ASR results and Phi-4 ASR results 

asr_rows = ['norm_whisper_asr', 'norm_phi4_asr', 'norm_parakeet', 'norm_granite']
align_ops_rows = ['norm_whisper_asr_ops', 'norm_phi4_asr_ops', 'norm_parakeet_ops', 'norm_granite_ops']

for asr_col, ops_col in zip(asr_rows, align_ops_rows):
    all_deletions = []
    all_insertions = []
    all_equals = []
    all_substitutions = []
    error_messages = []

    for index, row in evaluate_data.iterrows():
        try:
            result = extract_words_from_alignment(row['norm_human_transcript'], row[asr_col], row[ops_col])
            all_deletions.append(result[0])
            all_insertions.append(result[1])
            all_equals.append(result[2])
            all_substitutions.append(result[3])
            error_messages.append("")
        except Exception as e:
            all_deletions.append([])
            all_insertions.append([])
            all_equals.append([])
            all_substitutions.append([])
            error_messages.append(str(e))
            print(f"Error processing row {index} for {asr_col}: {str(e)}")

    # Creating DataFrame for the extracted words and error messages
    extracted_words_df = pd.DataFrame({
        f'{asr_col}_Deletions': all_deletions,
        f'{asr_col}_Insertions': all_insertions,
        # f'{asr_col}_Equals': all_equals,
        f'{asr_col}_Substitutions': all_substitutions,
        # f'{asr_col}_Error_Message': error_messages
    })

    # Concatenate the extracted words DataFrame with the original DataFrame
    evaluate_data = pd.concat([evaluate_data, extracted_words_df], axis=1, join='outer')


In [15]:
# apply alignment function to primock as well
primock_asr_rows = ['norm_whisper_doctor', 'norm_whisper_patient', 'norm_phi4_doctor', 'norm_phi4_patient',
                    'norm_parakeet_doctor', 'norm_parakeet_patient', 'norm_granite_doctor', 'norm_granite_patient']
primock_align_ops_rows = ['norm_whisper_doctor_ops', 'norm_whisper_patient_ops', 'norm_phi4_doctor_ops', 'norm_phi4_patient_ops', 'norm_parakeet_doctor_ops', 'norm_parakeet_patient_ops', 'norm_granite_doctor_ops', 'norm_granite_patient_ops']
for asr_col, ops_col in zip(primock_asr_rows, primock_align_ops_rows):
    all_deletions = []
    all_insertions = []
    all_equals = []
    all_substitutions = []
    error_messages = []

    for index, row in primock_merged.iterrows():
        try:
            result = extract_words_from_alignment(
                row['norm_human_doctor'] if 'doctor' in asr_col else row['norm_human_patient'],
                row[asr_col],
                row[ops_col]
            )
            all_deletions.append(result[0])
            all_insertions.append(result[1])
            all_equals.append(result[2])
            all_substitutions.append(result[3])
            error_messages.append("")
        except Exception as e:
            all_deletions.append([])
            all_insertions.append([])
            all_equals.append([])
            all_substitutions.append([])
            error_messages.append(str(e))
            print(f"Error processing row {index} for {asr_col}: {str(e)}")

    # Creating DataFrame for the extracted words and error messages
    extracted_words_df = pd.DataFrame({
        f'{asr_col}_Deletions': all_deletions,
        f'{asr_col}_Insertions': all_insertions,
        # f'{asr_col}_Equals': all_equals,
        f'{asr_col}_Substitutions': all_substitutions,
        # f'{asr_col}_Error_Message': error_messages
    })

    # Concatenate the extracted words DataFrame with the original DataFrame
    primock_merged = pd.concat([primock_merged, extracted_words_df], axis=1, join='outer')

## Reconstruct Human Transcript

In [8]:
## Reconstruct human transcript based on ASR outputs and alignment ops; color-code the errors encountered
def reconstruct_human_transcript(hyp_text, alignment_ops):
    reconstructed = []
    hyp_words = hyp_text.split() if isinstance(hyp_text, str) else []

    # if hypothesis empty -> return empty
    if not hyp_words:
        return ""

    processed = preprocess_alignment_ops(alignment_ops)

    # produce a flat list of op entries regardless of input shape
    if isinstance(processed, list):
        # flatten one level (handles [[...]] case)
        flat = []
        for item in processed:
            if isinstance(item, list):
                flat.extend(item)
            else:
                flat.append(item)
        alignment_ops_list = flat
    elif isinstance(processed, dict):
        alignment_ops_list = [processed]
    else:
        if not processed:
            return ""
        try:
            alignment_ops_list = ast.literal_eval(processed)
            # if parsed to nested list, flatten one level
            if isinstance(alignment_ops_list, list) and any(isinstance(x, list) for x in alignment_ops_list):
                flat = []
                for item in alignment_ops_list:
                    if isinstance(item, list):
                        flat.extend(item)
                    else:
                        flat.append(item)
                alignment_ops_list = flat
        except Exception as e:
            raise ValueError(f"Failed to parse alignment_ops: {e}")

    for op in alignment_ops_list:
        typ = _get_field(op, "type")
        if typ == "delete":
            s = _get_field(op, "ref_start_idx", 0)
            e = _get_field(op, "ref_end_idx", 0)
            deleted_words = f"[DEL: {' '.join(['__'+w+'__' for w in hyp_words[s:e]])}]"
            reconstructed.append(deleted_words)
        elif typ == "equal":
            s = _get_field(op, "hyp_start_idx", 0)
            e = _get_field(op, "hyp_end_idx", 0)
            reconstructed.extend(hyp_words[s:e])
        elif typ == "insert":
            s = _get_field(op, "hyp_start_idx", 0)
            e = _get_field(op, "hyp_end_idx", 0)
            inserted_words = f"[INS: {' '.join(['++'+w+'++' for w in hyp_words[s:e]])}]"
            reconstructed.append(inserted_words)
        elif typ == "substitute":
            s = _get_field(op, "ref_start_idx", 0)
            
            e = _get_field(op, "ref_end_idx", 0)
            substituted_words = f"[SUB: {' '.join(['~~'+w+'~~' for w in hyp_words[s:e]])}]"
            reconstructed.append(substituted_words)
    return ' '.join(reconstructed)

# apply reconstruct_human_transcript to each ASR column
for asr_col, ops_col in zip(asr_rows, align_ops_rows):
    evaluate_data[f'{asr_col}_reconstructed_human'] = evaluate_data.apply(
        lambda row: reconstruct_human_transcript(row['norm_human_transcript'], row[ops_col]), axis=1
    )

# # apply to whipser ASR only for now
# evaluate_data['norm_whisper_asr_reconstructed_human'] = evaluate_data.apply(
#     lambda row: reconstruct_human_transcript(row['norm_human_transcript'], row['norm_whisper_asr_ops']), axis=1
# )    

In [16]:
# reconstruct human transcript for primock as well
primock_asr_rows = ['norm_whisper_doctor', 'norm_whisper_patient', 'norm_phi4_doctor', 'norm_phi4_patient',
                    'norm_parakeet_doctor', 'norm_parakeet_patient', 'norm_granite_doctor', 'norm_granite_patient']
primock_align_ops_rows = ['norm_whisper_doctor_ops', 'norm_whisper_patient_ops', 'norm_phi4_doctor_ops', 'norm_phi4_patient_ops', 'norm_parakeet_doctor_ops', 'norm_parakeet_patient_ops', 'norm_granite_doctor_ops', 'norm_granite_patient_ops']
for asr_col, ops_col in zip(primock_asr_rows, primock_align_ops_rows):
    primock_merged[f'{asr_col}_reconstructed_human'] = primock_merged.apply(
        lambda row: reconstruct_human_transcript(
            row['norm_human_doctor'] if 'doctor' in asr_col else row['norm_human_patient'],
            row[ops_col]
        ), axis=1
    )

In [9]:
D: How can I uh, help you today?

P: Hi, so I just came in today because I've been just been feeling um, like my head's just been hurting for the past three or four days and I also felt really cold throughout the day for the last few days as well, so, um, I did end up checking my temperature just yesterday afternoon and it was um, 38.7 degrees celsius.

D: Okay, um. Tell me about the headache, where exactly is it? 

P: Yeah so it's kind of just throughout like the front of my forehead and it kind of goes to the side into my temple. 

D: And what does it feel like?

P: It feels like a constant aching. I've had uh, migraines before, but they like, it's not like the pulsating sensation that I usually get with them.

D: I see. Uh, okay, and how severe is the pain, if you had to rate it from 1 to 10?

P: Um, more than like severe. It's definitely less severe than my migraines, but it, it's just constant. It's, it's just been there for the last four days and I would say probably a 6 out of 10.

D: Did you do anything for the pain? Did you take any medications? Anything that makes it better?

P: Yeah, I took a Tylenol, I took um, Advil. It helps for like a few hours and then it just comes back.

D: And anything that makes the pain worse?

P: Um, I definitely think just, if I rest, it's better if, but if I'm like, like I can't, I can't concentrate on work or like anything.

D: Okay. Um, and other than the, um, the fever and headaches, have you um, had any um, cough?

P: No, no cough. 

D: Runny nose or congestion?

P: No.

D: Sore throat?

P: No, not really.

D: Changes in your sense of smell?

P: Uh, I do actually feel like I, like I am losing my smell today.

D: And sense of taste?

P: I just had breakfast today so um, breakfast was okay. I didn't, uh, but yeah, definitely sense of smell.

D: Okay, and any difficulties breathing?

P: No.

D: Okay, any chest pain?

P: No.

D: Um, do you feel like your heart is racing?

P: No. 

D: Any diarrhea or constipation?

P: No.

D: Nausea or vomiting?

P: No.

D: Pain in your belly?

P: No. 

D: Any muscle aches or joint pains?

P: Uh, no, nothing like that.

D: Okay, um, do you feel like your neck is stiff?

P: No. 

D: Any numbness or tingling in your arms or fingers, or toes?

P: No. 

D: Any um, night sweats or weight loss, unintentional?

P: No, I've only been gaining weight.

D: Okay. Any fluid in your, any sort of, swelling in your legs?

P: No, nothing like that really.

D: Okay, um, any medical conditions that you've been diagnosed with?

P: Yes, so I have uh, type 2 diabetes. I was just recently started on metformin medication.

D: When were you started on metformin?

P: So I think around uh, it's been around two months.

D: Okay, how is that going for you? Are you tolerating it okay?

P: Yeah I've, I've been tolerating it well. It's just they, they said like I'm kind of just on the edge of the diabetes, like my level, so they just want to act, um, proactively.

D: Yeah, that's good. Um, any other medical conditions?

P: No, that's, that's all.

D: Any medications other than metformin that you take? Anything over the counter?

P: Um, so I do take, when my migraines get bad, like I do take, I have taken naproxen before. 

D: Okay. Surgeries?

P: Uh, no surgeries.

D: Hospitalizations?

P: Uh no.

D: Okay, any allergies to medications, foods, or any environmental allergies?

P: Not to any medications, but I do have an allergy to peanuts.

D: Okay, what happens? 

P: I do get like a, an anaphylactic reaction. 

D: Okay, okay. Um, and any um, family history of any medical conditions?

P: Uh, so on my dad's side, they like all, like all the men seem to have diabetes, so it's good that I'm starting on the medication. Um on my mom's side, they're pretty healthy. I don't think my mom has any medical conditions.

D: Okay, good. Um, and do you currently live alone or do you live with other people?

P: Yeah, so I live with my wife, I live with my two kids, a son and daughter.

D: And um, have you or any of them had any exposures to anyone who could possibly be sick?

P: Um just, uh, so my wife is a school teacher, um, and they did have in-person. Just uh, she's a high school teacher, and they had some they have in-person classes. So yeah, I don't know, but she's been feeling well. My kids are feeling well too.

D: Okay, and have you traveled anywhere outside the province?

P: No.

D: Okay, and do you currently smoke or did you ever smoke in the past?

P: No, I've never smoked. I do smoke marijuana just, maybe once or twice a month with some friends.

D: Okay, any recreational drugs? 

P: Sorry, no. 

D: Um, any alcohol?

P: Yeah, I probably drink um, a couple of beers during the weekend.

D: Okay, got it. Um, alright. Well, given your symptoms, um, we would like to do a COVID swab, just in, or request a COVID swab to be done through public health, um, just to, just to rule it out. Your symptoms do sound like, or overlap with some of the symptoms that are seen in patients who do have COVID. It could be that you have um, just um, just I guess like your headache and fever. Um, I'll also like do a physical exam to see there's nothing else going on. But in the meantime, I, I'd like you to self isolate and stay away from the rest of your family members as well, um, at least until the results are out. And then when the results are out, if it's positive, you'll receive more guidelines from public health as to what you need to do.

P: Okay, sounds good, um, and I work from home, so I don't need to.

D: Oh that's good. Okay, so you work from home. 

P: Alright.

D: Good.

P: Thank you.

SyntaxError: invalid syntax (868003116.py, line 3)

In [ ]:
how can i [DEL: __uh__] uh help you [DEL: __hi__] today hi so i just came [DEL: __because__] in today because [DEL: __been__] ive [DEL: __um__] been just been feeling um like my heads just been hurting for the past three or four days and i also felt really cold throughout the day for the last [DEL: __um__] few days as well so um i did end up [DEL: __and__] checking my [DEL: __um__] temperature just yesterday afternoon and [INS: ++it++] was um 387 degrees celsius okay um tell me about the headache where exactly is [INS: ++it++] yeah so its kind of just throughout like the front of my forehead and it kind of [SUB: ~~temple~~] to the side into my temple and what does it feel like it feels [DEL: __uh__] like a constant aching ive had uh migraines before but they like its not like the pulsating sensation that [DEL: __uh__] i usually get with them i see uh okay and how [SUB: ~~rate~~] is the [SUB: ~~1~~] if [SUB: ~~10~~] had to rate it from 1 to 10 um more than like severe [DEL: __it__] its definitely less severe than my migraines but it its just constant its [SUB: ~~and~~] [DEL: __i__ __would__] just been [SUB: ~~a~~ ~~6~~] the last [DEL: __10__] four days [INS: ++and++ ++i++] would say probably a 6 [SUB: ~~did~~] [DEL: __you__] of 10 did you do anything for the pain did you take any medications anything [SUB: ~~um~~] makes [INS: ++it++] better yeah i took a tylenol i took um advil it helps for like a few hours and then it just comes back and anything that makes the pain worse [DEL: __if__] um i definitely think just if i rest its better if but if im [DEL: __anything__] like like i [INS: ++cant++] i cant concentrate on [INS: ++work++] or like anything [INS: ++okay++] um and other than the um the fever and headaches have you um had any um cough no no cough runny nose or congestion no sore throat [SUB: ~~uh~~] not [INS: ++really++] changes in your sense of [INS: ++smell++] uh i do [SUB: ~~losing~~] feel [SUB: ~~smell~~] i like [INS: ++i++] am losing my [INS: ++smell++] today and sense of taste [INS: ++i++] just [DEL: __um__] had breakfast today so um [SUB: ~~uh~~] was okay [INS: ++i++ ++didnt++] uh [INS: ++but++ ++yeah++] definitely sense of smell okay and any difficulties breathing no okay any chest pain [DEL: __um__] no um do you feel like your heart is racing no any diarrhea or constipation no nausea or vomiting no pain in your belly no any muscle aches or [DEL: __uh__] joint pains uh no nothing [DEL: __um__] like that okay um do you feel like your neck is stiff no any numbness or tingling in your arms or fingers [DEL: __um__] or toes no any um night sweats or [SUB: ~~only~~] loss unintentional no ive only been gaining weight okay any fluid in your any sort of [SUB: ~~nothing~~ ~~like~~ ~~that~~ ~~really~~] no [SUB: ~~um~~] like that really okay um any medical conditions that [DEL: __so__] youve been [DEL: __uh__] diagnosed with yes so i have uh type 2 diabetes i was [SUB: ~~were~~] recently [SUB: ~~started~~] on metformin medication when were you [DEL: __uh__] started on metformin so i think [SUB: ~~how~~] [DEL: __is__] uh its been around two months okay how is that going [DEL: __ive__] for you are you tolerating it okay [DEL: __they__] yeah ive ive been tolerating it well its just they they said like im kind of just on the edge of [DEL: __um__] the diabetes [INS: ++like++] my level so they just want to act um proactively yeah [INS: ++thats++] good [INS: ++um++] any other medical conditions no thats thats all any medications other than metformin that you take [INS: ++anything++] over the counter um so [SUB: ~~like~~] do take when my migraines get [INS: ++bad++] [SUB: ~~naproxen~~] i do [INS: ++take++] i [DEL: __uh__] have taken naproxen [DEL: __uh__] before okay surgeries uh no surgeries hospitalizations uh no okay any allergies to medications foods or any environmental allergies not to any medications but i do have an allergy to [DEL: __a__] peanuts okay what happens i [DEL: __um__] do get [DEL: __um__] like a an anaphylactic reaction okay [DEL: __uh__] okay um and any um [DEL: __they__ __like__] family [DEL: __like__ __all__] history of any medical conditions uh so on my dads side they like all like [DEL: __um__] all the men seem to have diabetes so its good that im starting on the medication [SUB: ~~okay~~] on [DEL: __um__] my moms side theyre pretty healthy i dont think my mom has any [SUB: ~~yeah~~] [DEL: __so__] conditions okay good um and do you currently live alone or do you live with other [DEL: __um__] people yeah so i live with my wife i live with my two kids a son [DEL: __um__] and [DEL: __uh__] daughter and um have you or any [DEL: __um__] of them had any exposures to [DEL: __uh__] anyone who could possibly be sick um just uh so my wife is a school teacher um and they did have inperson just uh shes a high school teacher and they had some they have inperson classes so yeah i dont know but shes been feeling well my kids are feeling well too okay and have you traveled anywhere outside the province no okay and do you currently smoke [SUB: ~~some~~] did you ever smoke in the past [DEL: __um__] no ive never [DEL: __i__] smoked [SUB: ~~drink~~] [DEL: __um__] do smoke [DEL: __of__] marijuana just maybe once or twice a [INS: ++month++ ++with++ ++some++] friends [SUB: ~~alright~~ ~~well~~ ~~given~~] drugs sorry no um any alcohol yeah i [INS: ++probably++] drink [SUB: ~~covid~~] a couple [SUB: ~~in~~] beers during the [SUB: ~~covid~~ ~~swab~~] [DEL: __to__] got it um alright [SUB: ~~health~~] given [INS: ++your++] symptoms um we would like to do [INS: ++a++] covid swab just in or request a covid swab to be done through public health um just to just to rule it out your symptoms do sound [DEL: __um__] like [DEL: __um__] or overlap with some of the symptoms that [DEL: __um__] are seen in patients who do have covid it could be that you have um just um just [DEL: __i__] i guess like your headache and fever um ill also like do a physical exam to see theres [DEL: __um__] nothing else going on but in the meantime i id like you to self isolate and stay away from the rest of your family members as well um at least until the results are [DEL: __um__] out and then when the results are out if its positive youll receive more [SUB: ~~so~~] from public health as [DEL: __alright__] to what you

## Save Files

In [11]:
# save the dataframe to a new excel file
evaluate_data.to_excel('all_result_processed.xlsx', index=False, engine='openpyxl')

In [12]:
# load the excel file and display the first few rows
df = pd.read_excel('all_result_processed.xlsx', engine='openpyxl')

# move each models' results to separate sheets in the excel file
with pd.ExcelWriter('all_result_separate_sheets.xlsx', engine='openpyxl') as writer:
    whisper_cols = ['utterance_id', 'human-transcript', 'Whisper-ASR', 'norm_human_transcript', 'norm_whisper_asr', 
                    'norm_whisper_asr_wer', 'norm_whisper_asr_ins', 'norm_whisper_asr_del', 
                    'norm_whisper_asr_sub', 'norm_whisper_asr_ops',
                    'norm_whisper_asr_Deletions', 'norm_whisper_asr_Insertions', 'norm_whisper_asr_Substitutions', 'norm_whisper_asr_reconstructed_human']
    phi4_cols = ['utterance_id', 'human-transcript', 'Phi-4-ASR', 'norm_human_transcript', 'norm_phi4_asr', 
                 'norm_phi4_asr_wer', 'norm_phi4_asr_ins', 'norm_phi4_asr_del', 
                 'norm_phi4_asr_sub', 'norm_phi4_asr_ops',
                 'norm_phi4_asr_Deletions', 'norm_phi4_asr_Insertions', 'norm_phi4_asr_Substitutions', 'norm_phi4_asr_reconstructed_human']
    parakeet_cols = ['utterance_id', 'human-transcript', 'Nvidia-Parakeet', 'norm_human_transcript', 'norm_parakeet', 
                     'norm_parakeet_wer', 'norm_parakeet_ins', 'norm_parakeet_del', 
                     'norm_parakeet_sub', 'norm_parakeet_ops',
                     'norm_parakeet_Deletions', 'norm_parakeet_Insertions', 'norm_parakeet_Substitutions', 'norm_parakeet_reconstructed_human']
    granite_cols = ['utterance_id', 'human-transcript', 'IBM-Granite', 'norm_human_transcript', 'norm_granite', 
                    'norm_granite_wer', 'norm_granite_ins', 'norm_granite_del', 
                    'norm_granite_sub', 'norm_granite_ops',
                    'norm_granite_Deletions', 'norm_granite_Insertions', 'norm_granite_Substitutions', 'norm_granite_reconstructed_human']
    df_whisper = df.filter(items=whisper_cols, axis=1)
    df_phi4 = df.filter(items=phi4_cols, axis=1)
    df_parakeet = df.filter(items=parakeet_cols, axis=1)
    df_granite = df.filter(items=granite_cols, axis=1)

    # Only write non-empty DataFrames to avoid invisible sheet error
    if not df_whisper.empty:
        df_whisper.to_excel(writer, sheet_name='Whisper-ASR Results', index=False)
    if not df_phi4.empty:
        df_phi4.to_excel(writer, sheet_name='Phi-4-ASR Results', index=False)
    if not df_parakeet.empty:
        df_parakeet.to_excel(writer, sheet_name='Nvidia-Parakeet Results', index=False)
    if not df_granite.empty:
        df_granite.to_excel(writer, sheet_name='IBM-Granite Results', index=False)
        
    # resize each row in the sheets to 120px
    for sheet_name in ['Whisper-ASR Results', 'Phi-4-ASR Results', 'Nvidia-Parakeet Results', 'IBM-Granite Results']:
        worksheet = writer.sheets.get(sheet_name)
        if worksheet:
            for row_idx in range(1, len(df) + 2):  # +2 to account for header row and 1-based indexing
                worksheet.row_dimensions[row_idx].height = 120


In [17]:
# seperate primock results into separate sheets as well
with pd.ExcelWriter('primock_result_separate_sheets.xlsx', engine='openpyxl') as writer:
    whisper_cols = ['conversation_id', 'turns', 'doctor_utterances', 'patient_utterances', 'norm_human_doctor', 'norm_human_patient', 'norm_whisper_doctor', 
                    'norm_whisper_patient', 'norm_whisper_doctor_wer', 'norm_whisper_doctor_ins', 'norm_whisper_doctor_del', 
                    'norm_whisper_doctor_sub', 'norm_whisper_doctor_ops',
                    'norm_whisper_doctor_Deletions', 'norm_whisper_doctor_Insertions', 'norm_whisper_doctor_Substitutions', 'norm_whisper_doctor_reconstructed_human',
                    'norm_whisper_patient_wer', 'norm_whisper_patient_ins', 'norm_whisper_patient_del', 
                    'norm_whisper_patient_sub', 'norm_whisper_patient_ops',
                    'norm_whisper_patient_Deletions', 'norm_whisper_patient_Insertions', 'norm_whisper_patient_Substitutions', 'norm_whisper_patient_reconstructed_human']
    phi4_cols = ['conversation_id', 'turns', 'doctor_utterances', 'patient_utterances', 'norm_human_doctor', 'norm_human_patient', 'norm_phi4_doctor', 
                 'norm_phi4_patient', 'norm_phi4_doctor_wer', 'norm_phi4_doctor_ins', 'norm_phi4_doctor_del', 
                 'norm_phi4_doctor_sub', 'norm_phi4_doctor_ops',
                 'norm_phi4_doctor_Deletions', 'norm_phi4_doctor_Insertions', 'norm_phi4_doctor_Substitutions', 'norm_phi4_doctor_reconstructed_human',
                 'norm_phi4_patient_wer', 'norm_phi4_patient_ins', 'norm_phi4_patient_del', 
                 'norm_phi4_patient_sub', 'norm_phi4_patient_ops',
                 'norm_phi4_patient_Deletions', 'norm_phi4_patient_Insertions', 'norm_phi4_patient_Substitutions', 'norm_phi4_patient_reconstructed_human']
    parakeet_cols = ['conversation_id', 'turns', 'doctor_utterances', 'patient_utterances', 'norm_human_doctor', 'norm_human_patient', 'norm_parakeet_doctor', 
                     'norm_parakeet_patient', 'norm_parakeet_doctor_wer', 'norm_parakeet_doctor_ins', 'norm_parakeet_doctor_del', 
                     'norm_parakeet_doctor_sub', 'norm_parakeet_doctor_ops',
                     'norm_parakeet_doctor_Deletions', 'norm_parakeet_doctor_Insertions', 'norm_parakeet_doctor_Substitutions', 'norm_parakeet_doctor_reconstructed_human',
                     'norm_parakeet_patient_wer', 'norm_parakeet_patient_ins', 'norm_parakeet_patient_del', 
                     'norm_parakeet_patient_sub', 'norm_parakeet_patient_ops',
                     'norm_parakeet_patient_Deletions', 'norm_parakeet_patient_Insertions', 'norm_parakeet_patient_Substitutions', 'norm_parakeet_patient_reconstructed_human']
    granite_cols = ['conversation_id', 'turns', 'doctor_utterances', 'patient_utterances', 'norm_human_doctor', 'norm_human_patient', 'norm_granite_doctor', 
                    'norm_granite_patient', 'norm_granite_doctor_wer', 'norm_granite_doctor_ins', 'norm_granite_doctor_del', 
                    'norm_granite_doctor_sub', 'norm_granite_doctor_ops',
                    'norm_granite_doctor_Deletions', 'norm_granite_doctor_Insertions', 'norm_granite_doctor_Substitutions', 'norm_granite_doctor_reconstructed_human',
                    'norm_granite_patient_wer', 'norm_granite_patient_ins', 'norm_granite_patient_del', 
                    'norm_granite_patient_sub', 'norm_granite_patient_ops',
                    'norm_granite_patient_Deletions', 'norm_granite_patient_Insertions', 'norm_granite_patient_Substitutions', 'norm_granite_patient_reconstructed_human']
    df_whisper = primock_merged.filter(items=whisper_cols, axis=1)
    df_phi4 = primock_merged.filter(items=phi4_cols, axis=1)
    df_parakeet = primock_merged.filter(items=parakeet_cols, axis=1)
    df_granite = primock_merged.filter(items=granite_cols, axis=1)
    if not df_whisper.empty:
        df_whisper.to_excel(writer, sheet_name='Whisper-ASR Results', index=False)
    if not df_phi4.empty:
        df_phi4.to_excel(writer, sheet_name='Phi-4-ASR Results', index=False)
    if not df_parakeet.empty:
        df_parakeet.to_excel(writer, sheet_name='Nvidia-Parakeet Results', index=False)
    if not df_granite.empty:
        df_granite.to_excel(writer, sheet_name='IBM-Granite Results', index=False)
    
# resize each row in the sheets to 120px
    for sheet_name in ['Whisper-ASR Results', 'Phi-4-ASR Results', 'Nvidia-Parakeet Results', 'IBM-Granite Results']:
        worksheet = writer.sheets.get(sheet_name)
        if worksheet:
            for row_idx in range(1, len(primock_merged) + 2):  # +2 to account for header row and 1-based indexing
                worksheet.row_dimensions[row_idx].height = 120
    
    
                
